In [22]:
import numpy as np

from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env

In [23]:
class QRPfRA_v3(MujocoEnv, utils.EzPickle):
    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file="/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA/IK_Models/qrpfra_v3_leg_ik_scene_left.xml", frame_skip=1, **kwargs):
        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            **kwargs,
        )

        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

        obs_size = self.data.qpos.size + self.data.qvel.size

        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64
        )

    def step(self, action):
        self.do_simulation(action, self.frame_skip)

        observation = self._get_obs()
        reward = 0
        info = {}

        if self.render_mode == "human":
            self.render()

        return observation, reward, False, False, info

    def _get_obs(self):
        #position = self.data.qpos.flat.copy()
        #velocity = self.data.qvel.flat.copy()
        sensor_data = self.data.sensordata.flat.copy()

        return sensor_data

    def reset_model(self):
        qpos = self.init_qpos
        qvel = self.init_qvel
        self.set_state(qpos, qvel)

        observation = self._get_obs()

        return observation

    def _get_reset_info(self):
        return {"works": True}



In [24]:
env = QRPfRA_v3()
check_env(env)
obs = env.reset()
print(obs)


(array([-0.046 ,  0.1435, -0.1245,  0.    ,  0.    ,  0.    ]), {'works': True})


In [25]:
leg_dataset = [] # [obs0, obs1, obs2, act0, act1, act2]

In [26]:
# Save the leg_dataset variable to a text file with comma separation
np.savetxt('leg_dataset.txt', leg_dataset, delimiter=',')


In [27]:
for i in range(-100, 101):
    for j in range(-100, 101):
        for k in range(-100, 101):
            #env.render_mode = "human"
            env_obs = []
            for step_cnt in range(0,3):
                obs, reward, done, _, info = env.step([float(m) for m in [i, j, k]])
                if step_cnt == 2:
                    env_obs = obs
            

            leg_dataset.append([env_obs[0], env_obs[1], env_obs[2], i/100, j/100, k/100])


env.close()

In [28]:
print(len(leg_dataset))

8120601


In [29]:
#Convert left leg list to numpy array
leg_dataset_as_np = np.array(leg_dataset)

In [30]:
print(leg_dataset_as_np[0:4])

observations = leg_dataset_as_np[:, 0:3][:]
print(observations)
actions = leg_dataset_as_np[:, 3:][:]
print(actions)

[[ 0.11055595 -0.03747343 -0.20882535 -1.         -1.         -1.        ]
 [ 0.13625602 -0.16099696 -0.12872141 -1.         -1.         -0.99      ]
 [ 0.13800882 -0.15550839 -0.13297578 -1.         -1.         -0.98      ]
 [ 0.13886801 -0.15284066 -0.13480302 -1.         -1.         -0.97      ]]
[[ 0.11055595 -0.03747343 -0.20882535]
 [ 0.13625602 -0.16099696 -0.12872141]
 [ 0.13800882 -0.15550839 -0.13297578]
 ...
 [ 0.03206231  0.04113233  0.0695648 ]
 [ 0.03152256  0.03991277  0.06925379]
 [ 0.03100701  0.03877662  0.06895672]]
[[-1.   -1.   -1.  ]
 [-1.   -1.   -0.99]
 [-1.   -1.   -0.98]
 ...
 [ 1.    1.    0.98]
 [ 1.    1.    0.99]
 [ 1.    1.    1.  ]]


In [40]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_obs, test_obs, train_actions, test_actions = train_test_split(observations, actions, test_size=0.1, shuffle=True)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(3,)),
    tf.keras.layers.Dense(3, activation='linear'),
    tf.keras.layers.Dense(32, activation='linear'),
    tf.keras.layers.Dense(64, activation='linear'),
    tf.keras.layers.Dense(32, activation='linear'),
    tf.keras.layers.Dense(3, activation='tanh')
])
# Compile the model
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

# Train the model
model.fit(observations, actions, epochs=5, batch_size=256)

# Evaluate the model on the testing set
test_loss = model.evaluate(test_obs, test_actions)
print('Test Loss:', test_loss)


Epoch 1/5
31722/31722 [==============================] - 20s 623us/step - loss: 0.1297 - accuracy: 0.7136
Epoch 2/5
31722/31722 [==============================] - 20s 631us/step - loss: 0.1294 - accuracy: 0.7139
Epoch 3/5
31722/31722 [==============================] - 20s 636us/step - loss: 0.1293 - accuracy: 0.7140
Epoch 4/5
31722/31722 [==============================] - 20s 618us/step - loss: 0.1293 - accuracy: 0.7141
Epoch 5/5
25377/25377 [==============================] - 7s 282us/step - loss: 0.1292 - accuracy: 0.7148
Test Loss: [0.12919948995113373, 0.7148268222808838]


In [ ]:
model.save("left_legs_model", overwrite=True)

INFO:tensorflow:Assets written to: left_legs_model/assets


INFO:tensorflow:Assets written to: left_legs_model/assets
